In [2]:
# import
import tensorflow as tf
from keras.utils import load_img
from keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
import pandas as pd

FileNotFoundError: [Errno 2] No such file or directory: 'row0.png'

## Feature Extraction from images on the folder paintings

In [ ]:
df = pd.DataFrame()
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
for row in range(120) :
    image = load_img('Paintings/row'+str(row)+'.png', target_size=(224, 224))
# convert the image pixels to a numpy array
    image = img_to_array(image)
# reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
    image = preprocess_input(image)
# get extracted features
    features = model.predict(image)
    df1 = pd.DataFrame(features)
    df =  pd.concat([df, df1], ignore_index=True)

users_df = pd.read_csv("users.txt", sep = "\t", dtype = str)
df["diag"]=users_df["diag"]
df.to_csv("CNNFeatureExtraction.csv")

## Classification

In [8]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

features_df = pd.DataFrame()
features_df = pd.read_csv("CNNFeatureExtraction.csv")
features_df = features_df.drop(["Unnamed: 0"], axis=1)

train, test = train_test_split(features_df, test_size=20)
trainY = train["diag"]
trainX = train.drop(["diag"], axis=1)
testY = test["diag"]
testX = test.drop(["diag"], axis=1)

trainX

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
41,0.0,0.000000,0.692729,0.235070,0.0,0.0,1.168126,0.0,0.0,0.0,...,0.0,0.0,0.571169,3.038742,0.0,0.000000,0.000000,0.0,0.0,1.770084
17,0.0,0.000000,1.904580,0.000000,0.0,0.0,1.317389,0.0,0.0,0.0,...,0.0,0.0,0.640355,1.750825,0.0,0.519510,0.661722,0.0,0.0,1.678769
63,0.0,0.000000,0.921763,0.000000,0.0,0.0,0.416031,0.0,0.0,0.0,...,0.0,0.0,0.725606,2.849451,0.0,0.000000,0.000000,0.0,0.0,1.684008
46,0.0,1.187201,1.414815,0.000000,0.0,0.0,0.390504,0.0,0.0,0.0,...,0.0,0.0,0.653767,2.343032,0.0,0.000000,0.000000,0.0,0.0,2.070436
110,0.0,0.981167,1.823059,0.073547,0.0,0.0,0.560495,0.0,0.0,0.0,...,0.0,0.0,1.173825,2.792661,0.0,0.008966,0.271342,0.0,0.0,1.828501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.605604,0.753420,0.000000,0.0,0.0,0.288876,0.0,0.0,0.0,...,0.0,0.0,1.346679,3.079764,0.0,0.000000,0.000000,0.0,0.0,2.043970
13,0.0,0.171703,2.628944,0.266749,0.0,0.0,1.268045,0.0,0.0,0.0,...,0.0,0.0,0.742406,2.216897,0.0,0.079677,0.170730,0.0,0.0,0.910821
60,0.0,0.000000,1.814274,0.000000,0.0,0.0,0.829988,0.0,0.0,0.0,...,0.0,0.0,1.012816,2.814088,0.0,0.000000,0.000000,0.0,0.0,1.029543
15,0.0,0.000000,1.448843,0.000000,0.0,0.0,1.916398,0.0,0.0,0.0,...,0.0,0.0,0.507560,2.497385,0.0,0.000000,0.000000,0.0,0.0,1.776090


In [9]:
ridModel = RidgeClassifier(alpha=100)
ridModel.fit(trainX, trainY)

print(ridModel.score(testX,testY))

0.85


In [10]:
logModel = LogisticRegression()
logModel.fit(trainX, trainY)

print(logModel.score(testX,testY))

0.85


C:\Users\Diego\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
tree = DecisionTreeClassifier()
tree.fit(trainX, trainY)
print(tree.score(testX,testY))

0.8


In [12]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(trainX,trainY)
print(clf.score(testX,testY))

0.85


### The accuracy is very variable therefore through this block it is possible to obtain the average results (now is set to 1000 classification, to change just change the number in the for)

In [15]:
import statistics
accRidge = []
accLogReg = []
accDecTree = []
accRnFr = []
ridModel = RidgeClassifier(alpha=100)
logModel = LogisticRegression()
tree = DecisionTreeClassifier()
clf=RandomForestClassifier(n_estimators=100)
for i in range(1000):
    train, test = train_test_split(features_df, test_size=20)
    trainY = train["diag"]
    trainX = train.drop(["diag"], axis=1)
    testY = test["diag"]
    testX = test.drop(["diag"], axis=1)


    ridModel.fit(trainX, trainY)
    accRidge.append(ridModel.score(testX, testY))
    logModel.fit(trainX, trainY)
    accLogReg.append(logModel.score(testX,testY))
    tree.fit(trainX, trainY)
    accDecTree.append(tree.score(testX,testY))
    clf.fit(trainX,trainY)
    accRnFr.append(clf.score(testX,testY))
print(statistics.mean(accRidge), statistics.mean(accLogReg), statistics.mean(accDecTree), statistics.mean(accRnFr))

C:\Users\Diego\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Diego\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

0.7754 0.77805 0.7049 0.72375


In [18]:
print("Media Acc RidgeClassifier: "+str(statistics.mean(accRidge)))
print("Accuracy piu elevata "+str(max(accRidge)))
print("Accuracy piu bassa "+str(min(accRidge)))
print("---------------------------------------------------")
print("Media Acc LogisticRegression: "+str(statistics.mean(accLogReg)))
print("Accuracy piu elevata "+str(max(accLogReg)))
print("Accuracy piu bassa "+str(min(accLogReg)))
print("---------------------------------------------------")
print("Media Acc DecisionTreeClassifier: "+str(statistics.mean(accDecTree)))
print("Accuracy piu elevata "+str(max(accDecTree)))
print("Accuracy piu bassa "+str(min(accDecTree)))
print("---------------------------------------------------")
print("Media Acc RandomForestClassifier: "+str(statistics.mean(accRnFr)))
print("Accuracy piu elevata "+str(max(accRnFr)))
print("Accuracy piu bassa "+str(min(accRnFr)))

Media Acc RidgeClassifier: 0.7754
Accuracy piu elevata1.0
Accuracy piu elevata0.45
---------------------------------------------------
Media Acc LogisticRegression: 0.77805
Accuracy piu elevata1.0
Accuracy piu elevata0.45
---------------------------------------------------
Media Acc DecisionTreeClassifier: 0.7049
Accuracy piu elevata1.0
Accuracy piu elevata0.3
---------------------------------------------------
Media Acc RandomForestClassifier: 0.72375
Accuracy piu elevata0.95
Accuracy piu elevata0.4
